In [56]:
!which python

/c/Users/valgo/Documents/rainbow-deployment/.venv/Scripts/python


In [57]:
import requests
import json
import pandas as pd
from urllib.error import HTTPError, URLError

# Complete dataspace flow

## Consts


In [58]:
hosp_pub_1 = "http://127.0.0.1:1200"
hosp_pub_2 = "http://127.0.0.1:1800"

hosp_priv= "http://127.0.0.1:1100"
api_endpoint = "https://datasets-server.huggingface.co/rows?dataset=ncbi%2FOpen-Patients&config=default&split=train&offset=0&length=100"
api_endpoint_p2 = "https://datasets-server.huggingface.co/rows?dataset=starmpcc%2FAsclepius-Synthetic-Clinical-Notes&config=default&split=train&offset=0&length=100"

selected_provider = "hosp_pub_1"

#provider 1
data_space_hosp_pub_1_participant_id = ""
data_space_hosp_pub_1_participant_token = ""
data_space_consumer_participant_id = ""
data_space_consumer_participant_token = ""
catalog_id = ""
dataset_id = ""
distribution_id = ""
data_service_id = ""
policy_id = ""
target_id = ""

# Provider 2
hosp_pub_2_participant_id = ""
hosp_pub_2_participant_token = ""
catalog_id_p2 = ""
dataset_id_p2 = ""
distribution_id_p2 = ""
data_service_id_p2 = ""
policy_id_p2 = ""
target_id_p2 = ""

# Consumer
cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""
consumer_data_address = ""

json_header = {
    "Content-Type": "application/json",
}

In [59]:
def replace_host_docker_internal(url: str) -> str:
    return url.replace("host.docker.internal", "127.0.0.1")

## Wallets setup

## Participants onboarding

### Current Participants information

In [60]:
url = hosp_pub_1+ "/api/v1/mates/myself"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiIwLW5uempxdDNQVXZtMUVPaEVxNUNxQ05WcEVUM3I4S0Y3VWRKSDJlRFV1dDNhSFlFWGFxbG51SXZ4TUVmN0J1dkhMVnlUaW95U1BoMV9TaDF0ZnBwdF9aNXpXU1ZZUGp0ekZ0QlJsQ0pQYzNfc2YtelRIQURMMmNyYW84WmxXa1FFR2ZtX1g3Y2QtMkd5bmVuR0k0a01rLU9vRERUWEJNRmVoOVdtR1dyMGZBeHJ4Qkp3Unh4QTExODNIOWo1QkY3dlpVZ0Y3akYzV28xcVlva0Z4emJ0Q2FhWWkzTmE3aFYtWWRIYkg3QUhkYnlJa1JvTHhhVG9tb3ktRHRmb0dSSGRzTmpzNDFqZUwxTG5XczJENnVJMm43QTBFQ1hIaHJjZTlMbkt6eXNMWktRM3Ffa2hxR0UzRTZJT1djajRMT1NvNGJ4SXktZzNVT0NpeElSelJ2bFEifQ",
    "participant_slug": "Myself",
    "participant_type": "Provider",
    "base_url": "http://host.docker.internal:1200",
    "token": null,
    "saved_at": "2025-12-13T18:20:26.887228",
    "last_interaction": "2025-12-13T18:20:26.887228",
    "is_me": true
}


In [61]:
url = hosp_pub_2 + "/api/v1/mates/myself"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiJxcW9lSmhkUnFEUGtxOUFZcDFkcVU3QkZNMlU3Y3FWVEdfV2g4NnNGTDNONlZGUnhrZ1doQzFFQmV1bnlHWWFTa3M1b0FsNWxjcmF4Snp5YXA4Q0pfd1M3N1Y0X1c5TU82VmZyWXpRLUpCb0FQRmdwWVZEbFUxYnJPZzJmZ0FqdkVZYTg3eFhwbUZmRjF3eE5SZWliWTNRczJZVUVhdjl4Q0JETkIxRTlHcHFTR3NkcUFUdHpTQW9PMGlrMWhhOTZJM1J6cmM4QXdqSGpGQzE0REJNRlo0b29JUmtNSUpPejRTc0lyMmlFbDBGSzE0MTY5aVhITWRITkIzUzBsY282bHpQTGlsa2I5bldJMzZzaDA1ZkdfOXktYUM3bG90eC1hTlczLVh1VDlqVl96ejNvc241YkZ2OU9mTllHVWdwM2UxU05jNmpkWnpVaXhzc3lFUXpOaFEifQ",
    "participant_slug": "Myself",
    "participant_type": "Provider",
    "base_url": "http://host.docker.internal:1800",
    "token": null,
    "saved_at": "2025-12-13T18:20:28.397719",
    "last_interaction": "2025-12-13T18:20:28.397720",
    "is_me": true
}


In [62]:
url = hosp_priv+ "/api/v1/mates/myself"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiJ4T0F3WlpkQ3AwQ0dCQV9tMVVHR1VXcEM5THJlcmJjbDMxblJoZThzNzd6R204aXA3YUgtUW5NNUNLbEo1YXlvUDhKTV9rLWhVQ0V2Q09qNWw5VmNhN0RfNTdRMjRJUEtHN0dQSGdnYzZ5bGxwZFlnSkVlY2xUcXJGcjdGUFBxcThDQzY1WVpfS1hGY3ByOHJGRVFEUGtkc3gzbU1qTkYzQUpNNzBpaEItdGtOOWxxcHZLb0h6MzJRaHRybVhFY2t1X1NtVFlFTU9GVU0xd0I1TE4xNmN4ZzIxZXYwMnZGdlVCYmdWZENTNHZocjNHNlNMM2dPZXVzd3BZaTZtTTZ6clloY3lkaDJaRjZCUEZVMS1NNk9tcENGRnd6SFVWNTdqMHJMLUR0OU1Ja1FUWmxXam5kcVVOZG1Ecnlyd0E1OU8zS1lWYWY5cmUwdW9QS3ZGUmRmY3cifQ",
    "participant_slug": "Myself",
    "participant_type": "Consumer",
    "base_url": "http://host.docker.internal:1100",
    "token": null,
    "saved_at": "2025-12-13T18:20:25.617584",
    "last_interaction": "2025-12-13T18:20:25.617585",
    "is_me": true
}


### Provider and Consumer ids

In [63]:
url = hosp_pub_1+ "/api/v1/mates/all"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Consumer":
        data_space_consumer_participant_id = participant["participant_id"]
        data_space_consumer_participant_token = participant["token"]
        break

print(json.dumps(response_as_json, indent=4))

[
    {
        "participant_id": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiIwLW5uempxdDNQVXZtMUVPaEVxNUNxQ05WcEVUM3I4S0Y3VWRKSDJlRFV1dDNhSFlFWGFxbG51SXZ4TUVmN0J1dkhMVnlUaW95U1BoMV9TaDF0ZnBwdF9aNXpXU1ZZUGp0ekZ0QlJsQ0pQYzNfc2YtelRIQURMMmNyYW84WmxXa1FFR2ZtX1g3Y2QtMkd5bmVuR0k0a01rLU9vRERUWEJNRmVoOVdtR1dyMGZBeHJ4Qkp3Unh4QTExODNIOWo1QkY3dlpVZ0Y3akYzV28xcVlva0Z4emJ0Q2FhWWkzTmE3aFYtWWRIYkg3QUhkYnlJa1JvTHhhVG9tb3ktRHRmb0dSSGRzTmpzNDFqZUwxTG5XczJENnVJMm43QTBFQ1hIaHJjZTlMbkt6eXNMWktRM3Ffa2hxR0UzRTZJT1djajRMT1NvNGJ4SXktZzNVT0NpeElSelJ2bFEifQ",
        "participant_slug": "Myself",
        "participant_type": "Provider",
        "base_url": "http://host.docker.internal:1200",
        "token": null,
        "saved_at": "2025-12-13T18:20:26.887228",
        "last_interaction": "2025-12-13T18:20:26.887228",
        "is_me": true
    },
    {
        "participant_id": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiJ4T0F3WlpkQ3AwQ0dCQV9tMVVHR1VXcEM5THJlcmJjbDMxblJoZThzNzd6R204aXA3YUgtUW5NNUNLbEo1Y

In [64]:
url = hosp_priv+ "/api/v1/mates/all"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Provider":
        data_space_hosp_pub_1_participant_id = participant["participant_id"]
        data_space_hosp_pub_1_participant_token = participant["token"]
        break

print(json.dumps(response_as_json, indent=4))

[
    {
        "participant_id": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiJ4T0F3WlpkQ3AwQ0dCQV9tMVVHR1VXcEM5THJlcmJjbDMxblJoZThzNzd6R204aXA3YUgtUW5NNUNLbEo1YXlvUDhKTV9rLWhVQ0V2Q09qNWw5VmNhN0RfNTdRMjRJUEtHN0dQSGdnYzZ5bGxwZFlnSkVlY2xUcXJGcjdGUFBxcThDQzY1WVpfS1hGY3ByOHJGRVFEUGtkc3gzbU1qTkYzQUpNNzBpaEItdGtOOWxxcHZLb0h6MzJRaHRybVhFY2t1X1NtVFlFTU9GVU0xd0I1TE4xNmN4ZzIxZXYwMnZGdlVCYmdWZENTNHZocjNHNlNMM2dPZXVzd3BZaTZtTTZ6clloY3lkaDJaRjZCUEZVMS1NNk9tcENGRnd6SFVWNTdqMHJMLUR0OU1Ja1FUWmxXam5kcVVOZG1Ecnlyd0E1OU8zS1lWYWY5cmUwdW9QS3ZGUmRmY3cifQ",
        "participant_slug": "Myself",
        "participant_type": "Consumer",
        "base_url": "http://host.docker.internal:1100",
        "token": null,
        "saved_at": "2025-12-13T18:20:25.617584",
        "last_interaction": "2025-12-13T18:20:25.617585",
        "is_me": true
    },
    {
        "participant_id": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiJtcHBQUnNOOURBSmZDNDNnaWNPSUZLTzBsT3l5d3lZSHZfcGZYWU4xNUhsRGVXOXZPZU9UVjBzTWdYUjZNM

## Create Catalog instances and metadata

In [65]:
url = hosp_pub_1+ "/api/v1/catalogs"
payload = {
    "foaf:homepage": "My dataspace catalog",
    "dct:title": "My dataspace catalog",

}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]
print(json.dumps(response_as_json, indent=4))

{
    "@context": [
        "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "Catalog",
    "@id": "urn:uuid:8bf4f1ce-003a-4f06-9385-87f5e684c5e9",
    "homepage": "My dataspace catalog",
    "theme": "",
    "keyword": "",
    "conformsTo": null,
    "creator": null,
    "identifier": "urn:uuid:8bf4f1ce-003a-4f06-9385-87f5e684c5e9",
    "issued": "2025-12-13T18:23:00.586487",
    "modified": null,
    "title": "My dataspace catalog",
    "description": [],
    "participantId": "urn:uuid:9df666cc-eefc-4242-b7bb-be77f4db6533",
    "hasPolicy": [],
    "extraFields": null,
    "catalog": [],
    "dataset": [],
    "service": []
}


In [66]:
url = hosp_pub_2 + "/api/v1/catalogs"
payload = {
    "foaf:homepage": "My dataspace catalog",
    "dct:title": "My dataspace catalog",

}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
catalog_id_p2 = response_as_json["@id"]
print(json.dumps(response_as_json, indent=4))

{
    "@context": [
        "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "Catalog",
    "@id": "urn:uuid:030eec36-6bad-4743-aa64-84b03b717d59",
    "homepage": "My dataspace catalog",
    "theme": "",
    "keyword": "",
    "conformsTo": null,
    "creator": null,
    "identifier": "urn:uuid:030eec36-6bad-4743-aa64-84b03b717d59",
    "issued": "2025-12-13T18:23:00.615911",
    "modified": null,
    "title": "My dataspace catalog",
    "description": [],
    "participantId": "urn:uuid:c2267454-80a9-4dca-8c23-974c5a37dec2",
    "hasPolicy": [],
    "extraFields": null,
    "catalog": [],
    "dataset": [],
    "service": []
}


In [67]:
url = hosp_pub_1+ "/api/v1/catalogs/{}/datasets".format(catalog_id)
payload = {
    "dct:title": "My dataspace dataset",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["@id"]
print(json.dumps(response_as_json, indent=4))

{
    "@context": [
        "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "Dataset",
    "@id": "urn:uuid:1b10934f-b54d-4cec-befd-c0c4c5d94a66",
    "theme": "",
    "keyword": "",
    "conformsTo": null,
    "creator": null,
    "identifier": "urn:uuid:1b10934f-b54d-4cec-befd-c0c4c5d94a66",
    "issued": "2025-12-13T18:23:00.646984",
    "modified": null,
    "title": "My dataspace dataset",
    "description": [],
    "hasPolicy": [],
    "extraFields": null,
    "distribution": []
}


In [68]:
url = hosp_pub_2 + "/api/v1/catalogs/{}/datasets".format(catalog_id_p2)
payload = {
    "dct:title": "My dataspace dataset",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id_p2 = response_as_json["@id"]
print(json.dumps(response_as_json, indent=4))

{
    "@context": [
        "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "Dataset",
    "@id": "urn:uuid:fde40918-3a3d-4023-9173-2c4ec4a8cc41",
    "theme": "",
    "keyword": "",
    "conformsTo": null,
    "creator": null,
    "identifier": "urn:uuid:fde40918-3a3d-4023-9173-2c4ec4a8cc41",
    "issued": "2025-12-13T18:23:00.683107",
    "modified": null,
    "title": "My dataspace dataset",
    "description": [],
    "hasPolicy": [],
    "extraFields": null,
    "distribution": []
}


In [69]:
url = hosp_pub_1+ "/api/v1/catalogs/{}/data-services".format(catalog_id)
payload = {
    "dcat:endpointURL": api_endpoint
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_service_id = response_as_json["@id"]
print(json.dumps(response_as_json, indent=4))

{
    "@context": [
        "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "DataService",
    "@id": "urn:uuid:7efc39ea-4b5d-4969-be45-0a7127eca4a4",
    "theme": "",
    "keyword": "",
    "endpointDescription": "",
    "endpointURL": "https://datasets-server.huggingface.co/rows?dataset=ncbi%2FOpen-Patients&config=default&split=train&offset=0&length=100",
    "conformsTo": null,
    "creator": null,
    "identifier": "urn:uuid:7efc39ea-4b5d-4969-be45-0a7127eca4a4",
    "issued": "2025-12-13T18:23:00.715603",
    "modified": null,
    "title": null,
    "description": [],
    "hasPolicy": [],
    "extraFields": null
}


In [70]:
url = hosp_pub_2+ "/api/v1/catalogs/{}/data-services".format(catalog_id_p2)
payload = {
    "dcat:endpointURL": api_endpoint_p2
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_service_id_p2 = response_as_json["@id"]
print(json.dumps(response_as_json, indent=4))

{
    "@context": [
        "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "DataService",
    "@id": "urn:uuid:d07e4c98-cd83-4d9b-bb4a-0b16a6b58bcf",
    "theme": "",
    "keyword": "",
    "endpointDescription": "",
    "endpointURL": "https://datasets-server.huggingface.co/rows?dataset=starmpcc%2FAsclepius-Synthetic-Clinical-Notes&config=default&split=train&offset=0&length=100",
    "conformsTo": null,
    "creator": null,
    "identifier": "urn:uuid:d07e4c98-cd83-4d9b-bb4a-0b16a6b58bcf",
    "issued": "2025-12-13T18:23:00.743756",
    "modified": null,
    "title": null,
    "description": [],
    "hasPolicy": [],
    "extraFields": null
}


In [71]:
url = hosp_pub_1+ "/api/v1/catalogs/{}/datasets/{}/distributions".format(catalog_id, dataset_id)
payload = {
    "dcat:accessService": data_service_id,
    "dct:formats": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
distribution_id = response_as_json["@id"]
print(json.dumps(response_as_json, indent=4))

{
    "@context": [
        "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "Distribution",
    "@id": "urn:uuid:34c952b3-6a96-470c-8d9d-8a4dada1fc29",
    "accessService": {
        "@context": [
            "https://w3id.org/dspace/2025/1/context.jsonld"
        ],
        "@type": "DataService",
        "@id": "urn:uuid:7efc39ea-4b5d-4969-be45-0a7127eca4a4",
        "theme": "",
        "keyword": "",
        "endpointDescription": "",
        "endpointURL": "https://datasets-server.huggingface.co/rows?dataset=ncbi%2FOpen-Patients&config=default&split=train&offset=0&length=100",
        "conformsTo": null,
        "creator": null,
        "identifier": "urn:uuid:7efc39ea-4b5d-4969-be45-0a7127eca4a4",
        "issued": "2025-12-13T18:23:00.715603",
        "modified": null,
        "title": null,
        "description": [],
        "hasPolicy": [],
        "extraFields": null
    },
    "identifier": "urn:uuid:34c952b3-6a96-470c-8d9d-8a4dada1fc29",
    "issued": "2

In [72]:
url = hosp_pub_2+ "/api/v1/catalogs/{}/datasets/{}/distributions".format(catalog_id_p2, dataset_id_p2)
payload = {
    "dcat:accessService": data_service_id_p2,
    "dct:formats": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
distribution_id_p2 = response_as_json["@id"]
print(json.dumps(response_as_json, indent=4))

{
    "@context": [
        "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "Distribution",
    "@id": "urn:uuid:6657fdac-fa75-4836-813e-c402a062375d",
    "accessService": {
        "@context": [
            "https://w3id.org/dspace/2025/1/context.jsonld"
        ],
        "@type": "DataService",
        "@id": "urn:uuid:d07e4c98-cd83-4d9b-bb4a-0b16a6b58bcf",
        "theme": "",
        "keyword": "",
        "endpointDescription": "",
        "endpointURL": "https://datasets-server.huggingface.co/rows?dataset=starmpcc%2FAsclepius-Synthetic-Clinical-Notes&config=default&split=train&offset=0&length=100",
        "conformsTo": null,
        "creator": null,
        "identifier": "urn:uuid:d07e4c98-cd83-4d9b-bb4a-0b16a6b58bcf",
        "issued": "2025-12-13T18:23:00.743756",
        "modified": null,
        "title": null,
        "description": [],
        "hasPolicy": [],
        "extraFields": null
    },
    "identifier": "urn:uuid:6657fdac-fa75-4836-813e-c402a0

## Create policies

In [73]:
url = hosp_pub_1+ "/api/v1/datasets/{}/policies".format(dataset_id)
payload = {
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }

    ],
    "obligation": [],
    "prohibition": []
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["@id"]
target_id = response_as_json["target"]
print(json.dumps(response_as_json, indent=4))

{
    "@id": "urn:uuid:b716ef5a-1344-4686-a11f-53b1d9b877a0",
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }
    ],
    "obligation": [],
    "@type": "Offer",
    "prohibition": [],
    "target": "urn:uuid:1b10934f-b54d-4cec-befd-c0c4c5d94a66"
}


In [74]:
url = hosp_pub_2+ "/api/v1/datasets/{}/policies".format(dataset_id_p2)
payload = {
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }

    ],
    "obligation": [],
    "prohibition": []
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id_p2 = response_as_json["@id"]
target_id_p2 = response_as_json["target"]
print(json.dumps(response_as_json, indent=4))


{
    "@id": "urn:uuid:5837019f-402a-4828-9c5c-84953cddeeec",
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }
    ],
    "obligation": [],
    "@type": "Offer",
    "prohibition": [],
    "target": "urn:uuid:fde40918-3a3d-4023-9173-2c4ec4a8cc41"
}


## Contract Negotiation

In [75]:
if selected_provider == "hosp_pub_1":
    selected_provider_participant_id = data_space_hosp_pub_1_participant_id
    selected_policy_id = policy_id
    selected_target_id = target_id
elif selected_provider == "hosp_pub_2":
    selected_provider_participant_id = hosp_pub_2_participant_id
    selected_policy_id = policy_id_p2
    selected_target_id = target_id_p2
else:
    raise ValueError("selected_provider must be 'hosp_pub_1' or 'hosp_pub_2'")

url = hosp_priv+ "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerParticipantId": selected_provider_participant_id,
    "offer": {
        "@id": selected_policy_id,    # policy id
        "target": selected_target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:66389988-034c-47f9-8326-b5b3f09f0715",
  "providerPid": "urn:uuid:090581e6-9a4a-430e-9d89-6837fcb18017",
  "offer": {
    "@id": "urn:uuid:b716ef5a-1344-4686-a11f-53b1d9b877a0",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:1b10934f-b54d-4cec-befd-c0c4c5d94a66"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:090581e6-9a4a-430e-9d89-6837fcb18017",
    "consumerPid": "urn:uuid:66389988-034c-47f9-8326-b5b3f09f0715",
    "state": "REQUESTED"
  }
}


In [76]:
url = hosp_priv+ "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_hosp_pub_1_participant_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:46b9d7bd-1149-4ac7-84f8-f523a1a9a8fc",
  "providerPid": "urn:uuid:47322485-972f-4e92-8410-275177a51db0",
  "offer": {
    "@id": "urn:uuid:b716ef5a-1344-4686-a11f-53b1d9b877a0",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:1b10934f-b54d-4cec-befd-c0c4c5d94a66"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:47322485-972f-4e92-8410-275177a51db0",
    "consumerPid": "urn:uuid:46b9d7bd-1149-4ac7-84f8-f523a1a9a8fc",
    "state": "REQUESTED"
  }
}


In [77]:
url = hosp_pub_1+ "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id, # remove to test offer from provider
    "providerPid": cn_provider_id, # remove to test offer from provider
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerParticipantId": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiJ4T0F3WlpkQ3AwQ0dCQV9tMVVHR1VXcEM5THJlcmJjbDMxblJoZThzNzd6R204aXA3YUgtUW5NNUNLbEo1YXlvUDhKTV9rLWhVQ0V2Q09qNWw5VmNhN0RfNTdRMjRJUEtHN0dQSGdnYzZ5bGxwZFlnSkVlY2xUcXJGcjdGUFBxcThDQzY1WVpfS1hGY3ByOHJGRVFEUGtkc3gzbU1qTkYzQUpNNzBpaEItdGtOOWxxcHZLb0h6MzJRaHRybVhFY2t1X1NtVFlFTU9GVU0xd0I1TE4xNmN4ZzIxZXYwMnZGdlVCYmdWZENTNHZocjNHNlNMM2dPZXVzd3BZaTZtTTZ6clloY3lkaDJaRjZCUEZVMS1NNk9tcENGRnd6SFVWNTdqMHJMLUR0OU1Ja1FUWmxXam5kcVVOZG1Ecnlyd0E1OU8zS1lWYWY5cmUwdW9QS3ZGUmRmY3cifQ",
  "consumerPid": "urn:uuid:46b9d7bd-1149-4ac7-84f8-f523a1a9a8fc",
  "providerPid": "urn:uuid:47322485-972f-4e92-8410-275177a51db0",
  "offer": {
    "@id": "urn:uuid:b716ef5a-1344-4686-a11f-53b1d9b877a0",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:1b10934f-b54d-4cec-befd-c0c4c5d94a66"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    

In [78]:
url = hosp_priv+ "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_hosp_pub_1_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:46b9d7bd-1149-4ac7-84f8-f523a1a9a8fc",
  "providerPid": "urn:uuid:47322485-972f-4e92-8410-275177a51db0",
  "offer": {
    "@id": "urn:uuid:b716ef5a-1344-4686-a11f-53b1d9b877a0",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:1b10934f-b54d-4cec-befd-c0c4c5d94a66"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:47322485-972f-4e92-8410-275177a51db0",
    "consumerPid": "urn:uuid:46b9d7bd-1149-4ac7-84f8-f523a1a9a8fc",
    "state": "REQUESTED"
  }
}


In [79]:
url = hosp_pub_1+ "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerParticipantId": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiJ4T0F3WlpkQ3AwQ0dCQV9tMVVHR1VXcEM5THJlcmJjbDMxblJoZThzNzd6R204aXA3YUgtUW5NNUNLbEo1YXlvUDhKTV9rLWhVQ0V2Q09qNWw5VmNhN0RfNTdRMjRJUEtHN0dQSGdnYzZ5bGxwZFlnSkVlY2xUcXJGcjdGUFBxcThDQzY1WVpfS1hGY3ByOHJGRVFEUGtkc3gzbU1qTkYzQUpNNzBpaEItdGtOOWxxcHZLb0h6MzJRaHRybVhFY2t1X1NtVFlFTU9GVU0xd0I1TE4xNmN4ZzIxZXYwMnZGdlVCYmdWZENTNHZocjNHNlNMM2dPZXVzd3BZaTZtTTZ6clloY3lkaDJaRjZCUEZVMS1NNk9tcENGRnd6SFVWNTdqMHJMLUR0OU1Ja1FUWmxXam5kcVVOZG1Ecnlyd0E1OU8zS1lWYWY5cmUwdW9QS3ZGUmRmY3cifQ",
  "consumerPid": "urn:uuid:46b9d7bd-1149-4ac7-84f8-f523a1a9a8fc",
  "providerPid": "urn:uuid:47322485-972f-4e92-8410-275177a51db0",
  "offer": {
    "@id": "urn:uuid:b716ef5a-1344-4686-a11f-53b1d9b877a0",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:1b10934f-b54d-4cec-befd-c0c4c5d94a66"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    

In [80]:
url = hosp_priv+ "/api/v1/negotiations/rpc/setup-acceptance"
payload = {
    "providerParticipantId": data_space_hosp_pub_1_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:46b9d7bd-1149-4ac7-84f8-f523a1a9a8fc",
  "providerPid": "urn:uuid:47322485-972f-4e92-8410-275177a51db0",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:47322485-972f-4e92-8410-275177a51db0",
    "consumerPid": "urn:uuid:46b9d7bd-1149-4ac7-84f8-f523a1a9a8fc",
    "state": "ACCEPTED"
  }
}


In [81]:
url = hosp_pub_1+ "/api/v1/negotiations/rpc/setup-agreement"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
agreement_id=response_as_json["agreement"]["@id"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:46b9d7bd-1149-4ac7-84f8-f523a1a9a8fc",
  "providerPid": "urn:uuid:47322485-972f-4e92-8410-275177a51db0",
  "agreement": {
    "@id": "urn:uuid:86584e78-9fee-4ae2-a8ef-00565c0d8a39",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Agreement",
    "target": "urn:uuid:1b10934f-b54d-4cec-befd-c0c4c5d94a66",
    "assigner": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiIwLW5uempxdDNQVXZtMUVPaEVxNUNxQ05WcEVUM3I4S0Y3VWRKSDJlRFV1dDNhSFlFWGFxbG51SXZ4TUVmN0J1dkhMVnlUaW95U1BoMV9TaDF0ZnBwdF9aNXpXU1ZZUGp0ekZ0QlJsQ0pQYzNfc2YtelRIQURMMmNyYW84WmxXa1FFR2ZtX1g3Y2QtMkd5bmVuR0k0a01rLU9vRERUWEJNRmVoOVdtR1dyMGZBeHJ4Qkp3Unh4QTExODNIOWo1QkY3dlpVZ0Y3akYzV28xcVlva0Z4emJ0Q2FhWWkzTmE3aFYtWWRIYkg3QUhkYnlJa1JvTHhhVG9tb3ktRHRmb0dSSGRzTmpzNDFqZUwxTG5XczJENnVJMm43QTBFQ1hIaHJjZTlMbkt6eXNMWktRM3Ffa2hxR0UzRTZJT1djajRMT1NvNGJ4SXktZzNVT0NpeElSelJ2bFEifQ",
    "assignee": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiJ4T0F3WlpkQ3AwQ0dCQV9tMVVHR1VXcEM5T

In [82]:
url = hosp_priv+ "/api/v1/negotiations/rpc/setup-verification"
payload = {
    "providerParticipantId": data_space_hosp_pub_1_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:46b9d7bd-1149-4ac7-84f8-f523a1a9a8fc",
  "providerPid": "urn:uuid:47322485-972f-4e92-8410-275177a51db0",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:47322485-972f-4e92-8410-275177a51db0",
    "consumerPid": "urn:uuid:46b9d7bd-1149-4ac7-84f8-f523a1a9a8fc",
    "state": "VERIFIED"
  }
}


In [83]:
url = hosp_pub_1+ "/api/v1/negotiations/rpc/setup-finalization"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:46b9d7bd-1149-4ac7-84f8-f523a1a9a8fc",
  "providerPid": "urn:uuid:47322485-972f-4e92-8410-275177a51db0",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:47322485-972f-4e92-8410-275177a51db0",
    "consumerPid": "urn:uuid:46b9d7bd-1149-4ac7-84f8-f523a1a9a8fc",
    "state": "FINALIZED"
  }
}


## Transfer Negotiation


In [84]:
url = hosp_priv+ "/api/v1/transfers/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_hosp_pub_1_participant_id,
    "agreementId": agreement_id,
    "format": "http+pull",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_consumer_id = response_as_json["consumerPid"]
transfer_provider_id = response_as_json["providerPid"]
callback_address = response_as_json["callbackAddress"]
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:5ca01c49-2a59-43a8-8d75-c80799b44e7e",
  "consumerPid": "urn:uuid:95d7d616-5cb8-420c-81e6-7f145e69ae4f",
  "agreementId": "urn:uuid:86584e78-9fee-4ae2-a8ef-00565c0d8a39",
  "callbackAddress": "http://host.docker.internal:1100/urn:uuid:66641ba2-b358-4471-aa43-d2da92840db1",
  "format": "http+pull",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:5ca01c49-2a59-43a8-8d75-c80799b44e7e",
    "consumerPid": "urn:uuid:95d7d616-5cb8-420c-81e6-7f145e69ae4f",
    "state": "REQUESTED"
  }
}


In [85]:
url = hosp_pub_1+ "/api/v1/transfers/rpc/setup-start"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:5ca01c49-2a59-43a8-8d75-c80799b44e7e",
  "consumerPid": "urn:uuid:95d7d616-5cb8-420c-81e6-7f145e69ae4f",
  "dataAddress": {
    "@type": "DataAddress",
    "endpointType": "http",
    "endpoint": "http://host.docker.internal:1200/data/urn:uuid:5ca01c49-2a59-43a8-8d75-c80799b44e7e",
    "endpointProperties": [
      {
        "@type": "EndpointProperty",
        "name": "authType",
        "value": ""
      },
      {
        "@type": "EndpointProperty",
        "name": "authorization",
        "value": ""
      }
    ]
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:5ca01c49-2a59-43a8-8d75-c80799b44e7e",
    "consumerPid": "urn:uuid:95d7d616-5cb8-420c-81e6-7f145e69ae4f",
    "state": "STARTED"
  }
}


In [86]:
url = hosp_priv+ "/api/v1/dataplane/" + transfer_consumer_id
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
consumer_data_address = replace_host_docker_internal(response_as_json["process_address"]["url"])
print(json.dumps(response_as_json, indent=2))

{
  "id": "urn:uuid:95d7d616-5cb8-420c-81e6-7f145e69ae4f",
  "process_direction": "PULL",
  "upstream_hop": {
    "protocol": "",
    "url": "",
    "auth_type": "",
    "auth_content": ""
  },
  "downstream_hop": {
    "protocol": "",
    "url": "",
    "auth_type": "",
    "auth_content": ""
  },
  "process_address": {
    "protocol": "http",
    "url": "http://host.docker.internal:1100/data/urn:uuid:95d7d616-5cb8-420c-81e6-7f145e69ae4f",
    "auth_type": "",
    "auth_content": ""
  },
  "created_at": "2025-12-13T18:23:01.563542",
  "updated_at": "2025-12-13T18:23:01.597930",
  "state": "STARTED"
}


In [87]:
try:
    response = requests.get(consumer_data_address)
    data = response.json()
    
    # Si tiene estructura de Hugging Face con 'rows'
    if 'rows' in data:
        df = pd.DataFrame(data['rows'])
    else:
        df = pd.DataFrame(data)
    
    print(f"✓ {len(df)} registros de pacientes")
    print(df.head())
    
except Exception as e:
    print(f"Error: {e}")


✓ 100 registros de pacientes
   row_idx                                                row truncated_cells
0        0  {'_id': 'trec-cds-2014-1', 'description': 'A 5...              []
1        1  {'_id': 'trec-cds-2014-2', 'description': 'An ...              []
2        2  {'_id': 'trec-cds-2014-3', 'description': 'A 5...              []
3        3  {'_id': 'trec-cds-2014-4', 'description': 'A 2...              []
4        4  {'_id': 'trec-cds-2014-5', 'description': 'A 5...              []


In [88]:
url = hosp_priv+ "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "providerParticipantId": data_space_hosp_pub_1_participant_id,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:5ca01c49-2a59-43a8-8d75-c80799b44e7e",
  "consumerPid": "urn:uuid:95d7d616-5cb8-420c-81e6-7f145e69ae4f",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:5ca01c49-2a59-43a8-8d75-c80799b44e7e",
    "consumerPid": "urn:uuid:95d7d616-5cb8-420c-81e6-7f145e69ae4f",
    "state": "SUSPENDED"
  }
}


In [89]:
try:
    response = requests.get(consumer_data_address)
    data = response.json()
    
    # Si tiene estructura de Hugging Face con 'rows'
    if 'rows' in data:
        df = pd.DataFrame(data['rows'])
    else:
        df = pd.DataFrame(data)
    
    print(f"✓ {len(df)} registros de pacientes")
    print(df.head())
    
except Exception as e:
    print(f"Error: {e}")

Error: Expecting value: line 1 column 1 (char 0)


In [90]:
url = hosp_priv+ "/api/v1/transfers/rpc/setup-start"
payload = {
    "providerParticipantId": data_space_hosp_pub_1_participant_id,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:5ca01c49-2a59-43a8-8d75-c80799b44e7e",
  "consumerPid": "urn:uuid:95d7d616-5cb8-420c-81e6-7f145e69ae4f",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:5ca01c49-2a59-43a8-8d75-c80799b44e7e",
    "consumerPid": "urn:uuid:95d7d616-5cb8-420c-81e6-7f145e69ae4f",
    "state": "STARTED"
  }
}


In [91]:
try:
    response = requests.get(consumer_data_address)
    data = response.json()
    
    # Si tiene estructura de Hugging Face con 'rows'
    if 'rows' in data:
        df = pd.DataFrame(data['rows'])
    else:
        df = pd.DataFrame(data)
    
    print(f"✓ {len(df)} registros de pacientes")
    print(df.head())
    
except Exception as e:
    print(f"Error: {e}")

✓ 100 registros de pacientes
   row_idx                                                row truncated_cells
0        0  {'_id': 'trec-cds-2014-1', 'description': 'A 5...              []
1        1  {'_id': 'trec-cds-2014-2', 'description': 'An ...              []
2        2  {'_id': 'trec-cds-2014-3', 'description': 'A 5...              []
3        3  {'_id': 'trec-cds-2014-4', 'description': 'A 2...              []
4        4  {'_id': 'trec-cds-2014-5', 'description': 'A 5...              []


In [92]:
url = hosp_pub_1+ "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:5ca01c49-2a59-43a8-8d75-c80799b44e7e",
  "consumerPid": "urn:uuid:95d7d616-5cb8-420c-81e6-7f145e69ae4f",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:5ca01c49-2a59-43a8-8d75-c80799b44e7e",
    "consumerPid": "urn:uuid:95d7d616-5cb8-420c-81e6-7f145e69ae4f",
    "state": "SUSPENDED"
  }
}


In [93]:
url = hosp_pub_1+ "/api/v1/transfers/rpc/setup-completion"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:5ca01c49-2a59-43a8-8d75-c80799b44e7e",
  "consumerPid": "urn:uuid:95d7d616-5cb8-420c-81e6-7f145e69ae4f",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:5ca01c49-2a59-43a8-8d75-c80799b44e7e",
    "consumerPid": "urn:uuid:95d7d616-5cb8-420c-81e6-7f145e69ae4f",
    "state": "COMPLETED"
  }
}
